In [ ]:
!pip install streamlit -q

     |████████████████████████████████| 9.2 MB 8.1 MB/s 
     |████████████████████████████████| 4.7 MB 51.2 MB/s 
     |████████████████████████████████| 78 kB 6.5 MB/s 
     |████████████████████████████████| 164 kB 47.2 MB/s 
     |████████████████████████████████| 237 kB 53.1 MB/s 
     |████████████████████████████████| 182 kB 45.7 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 51 kB 6.0 MB/s 


In [ ]:
%%writefile app.py
import streamlit as st

import pandas as pd

#Setting Title of App
st.title("Plant Disease Prediction")
st.markdown("Upload an image of the plant leaf")

#Uploading the plant image
plant_image = st.file_uploader("Choose an image...", type="jpg")
submit = st.button('Predict')
#On predict button click

#ngrok

Writing app.py


In [ ]:
import numpy as np
import os
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
import cv2
from PIL import Image

In [ ]:
INIT_LR = 1e-3
BATCH_SIZE = 32
EPOCHS = 50
IMAGE_SIZE = 256
default_image_size = tuple((IMAGE_SIZE, IMAGE_SIZE))
image_size = 0
data_dir = "/content/drive/MyDrive/Potato_Leaves_Dataset"
# /content/drive/MyDrive/Potato_Disease_Predict.ipynb
CHANNELS=3
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1

    ds_size = ds.cardinality().numpy()

    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    return train_ds, val_ds, test_ds

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=default_image_size,
  batch_size=BATCH_SIZE
)
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

Found 2152 files belonging to 3 classes.


In [ ]:
%%writefile /content/drive/MyDrive/Potato_Disease_Codes/New_app.py
#/content/drive/MyDrive/Potato_Disease_Codes/New_app.py
import numpy as np
import os
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
import cv2
from PIL import Image

INIT_LR = 1e-3
BATCH_SIZE = 32
EPOCHS = 50
IMAGE_SIZE = 256
default_image_size = tuple((IMAGE_SIZE, IMAGE_SIZE))
image_size = 0
data_dir = "/content/drive/MyDrive/Potato_Leaves_Dataset"
# /content/drive/MyDrive/Potato_Disease_Predict.ipynb
CHANNELS=3
AUTOTUNE = tf.data.AUTOTUNE

def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1

    ds_size = ds.cardinality().numpy()

    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    return train_ds, val_ds, test_ds

dataset = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=default_image_size,
  batch_size=BATCH_SIZE
)
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)
#######################################################################################

import streamlit as st
file_dir='/content/drive/MyDrive/TestImage_Potatoes'
#Read and display the banner
img_1_path= '/content/drive/MyDrive/TestImage_Potatoes/test_img1.JPG'
img_2_path= '/content/drive/MyDrive/TestImage_Potatoes/test_img2.JPG'
img_3_path= '/content/drive/MyDrive/TestImage_Potatoes/test_img3.JPG'
banner_path= '/content/drive/MyDrive/Potato_Disease_Codes/Potato-Disease-Logo-1.png'
st.sidebar.image(banner_path,use_column_width=True)

#App Title
st.title("Potato Plant Disease Prediction - WebApp")

#App description
st.header("The app predicts diseases in Potato plants.")
st.subheader("The model was trained with the [potato leaf disease dataset on Kaggle](https://www.kaggle.com/arjuntejaswi/plant-village). ")

st.markdown('***')

#Load the saved model
model_saved = tf.keras.models.load_model("/content/drive/MyDrive/Potato_Disease_Codes/potato_model.h5")
class_names = ['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']


#Set the selectbox for demo images
st.write('**Select an image for a DEMO**')
menu = ['Select an Image','Image 1', 'Image 2', 'Image 3']
choice = st.selectbox('Select an image', menu)


#Set the box for the user to upload an image
st.write("**Upload your Image**")
uploaded_image = st.file_uploader("Upload your image in JPG or PNG format", type=["jpg", "png"])

#Set red flag if no image is selected/uploaded
if uploaded_image is None and choice=='Select an Image':
    st.sidebar.error("**STATUS:** IMAGE NOT UPLOADED")
    st.sidebar.warning("**Please Upload or Select an Image**")

def deploy(img_path,demo=True):
  st.sidebar.success("**STATUS**: Image uploaded successfully!")
  st.sidebar.image(img_path, width=301, channels='BGR')

  img_bgr = cv2.imread(filename=img_path)
  img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
  img_tensor = tf.convert_to_tensor(img_rgb, dtype=tf.float32)
  img_resized = tf.image.resize_with_pad(img_tensor, 256, 256)
  img_final = tf.expand_dims(img_resized, 0)
  st.image(img_path)
  predictions = model_saved.predict(img_final)
  st.write("Predictions:",predictions)
  pred_class = class_names[np.argmax(predictions[0])]
  st.subheader(f"Predicted Potato Class: {pred_class}")

  st.sidebar.info(f"**{pred_class}**")

#Deploy the model if the user uploads an image
if uploaded_image is not None:
    #Close the demo
    choice='Select an Image'
    #Deploy the model with the uploaded image
    #Saving upload
    with open(os.path.join(file_dir,uploaded_image.name),"wb") as f:
      f.write((uploaded_image).getbuffer())
    image_dir=file_dir+'/'+uploaded_image.name
    deploy(image_dir, demo=False)
    del uploaded_image

#Deploy the model if the user selects Image 1
if choice== 'Image 1':
    deploy(img_1_path)

#Deploy the model if the user selects Image 2
if choice== 'Image 2':
    deploy(img_2_path)

#Deploy the model if the user selects Image 3
if choice== 'Image 3':
    deploy(img_3_path)

Overwriting /content/drive/MyDrive/Potato_Disease_Codes/New_app.py


In [ ]:
!streamlit run /content/drive/MyDrive/Potato_Disease_Codes/New_app.py & npx localtunnel --port 8501

2022-10-10 04:45:01.535 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
[##................] | loadDep:strip-ansi: sill resolveWithNewModule wrap-ansi@


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.185.247.211:8501

npx: installed 22 in 7.227s
your url is: https://silent-lies-pull-35-185-247-211.loca.lt
Found 2152 files belonging to 3 classes.
2022-10-10 04:45:35.394752: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Found 2152 files belonging to 3 classes.
Found 2152 files belonging to 3 classes.
Found 2152 files belonging to 3 classes.
Found 2152 files belonging to 3 classes.
Found 2152 files belonging to 3 classes.
Found 2152 files belonging to 3 classes.
Found 2152 files belonging to 3 classes.
Found 2152 files belonging to 3 classes.
2022-10-10 05:20:35.035 5 out of the last 5 calls to <function Model.make_predic